In [2]:
import findspark  # this needs to be the first import                                                                        
findspark.init()

# jupyter notebook &

from pyspark.sql import DataFrame
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

import numpy as np
import pandas as pd
# import scipy as sc

from IPython.display import display
# pip install graphlab-create
import graphlab as gl
import json

gl.canvas.set_target('ipynb')

In [3]:
sc 

<SparkContext master=yarn appName=PySparkShell>

In [4]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
try:
    sc = SparkContext()     
except:
    pass
# spark = SparkSession(sc)

In [5]:
sc

<SparkContext master=yarn appName=PySparkShell>

In [6]:
# del spark
# spark = SparkSession \
#         .builder \
#         .master("yarn") \
#         .appName("testing") \
#         .config("spark.executor.instances", "4") \
#         .config("spark.executor.memory","2g") \
#         .config("spark.driver.memory","4g") \
#         .config("spark.executor.cores",'1') \
#         .config("spark.scheduler.mode","FIFO") \
#        .getOrCreate()

In [7]:
spark

In [8]:
# Actually, why should I partition?
dat = spark.read.json('hdfs://localhost:9000/yelp/checkin.json').repartition(150)

In [9]:
%%time
# count num of unique rows in a column
dat.select(col("time")).distinct().count()

CPU times: user 5.3 ms, sys: 2.28 ms, total: 7.59 ms
Wall time: 25.7 s


120782

In [10]:
%%time
# count num of unique rows in a column
dat.select(col("time")).distinct().count()

CPU times: user 4.49 ms, sys: 1.92 ms, total: 6.41 ms
Wall time: 17.5 s


120782

In [11]:
bus = spark.read.json('hdfs://localhost:9000/yelp/business.json').repartition(150)
# bus.printSchema()

# phot = spark.read.json('hdfs://localhost:9000/yelp/photos.json').repartition(150)
# phot.printScheme()

review= spark.read.json('hdfs://localhost:9000/yelp/review.json').repartition(150)
# phot.printScheme()

In [12]:
import time

In [13]:
_yyy = review.join(bus, review.business_id == bus.business_id, 'inner').drop(bus.business_id).drop(bus.stars)
# join based on user and business keys
# other objects get garbage collected

# _yyy.select("stars")
review_business = _yyy.withColumnRenamed("stars", "business_avg_stars").withColumnRenamed("attributes", "business_type").withColumnRenamed(\
                                         "review_count", "business_review_count")

In [14]:
from sys import getsizeof
getsizeof(review_business)

64

In [13]:
review_business.columns

['business_id',
 'cool',
 'date',
 'funny',
 'review_id',
 'business_avg_stars',
 'text',
 'useful',
 'user_id',
 'address',
 'business_type',
 'categories',
 'city',
 'hours',
 'is_open',
 'latitude',
 'longitude',
 'name',
 'neighborhood',
 'postal_code',
 'business_review_count',
 'state']

In [14]:
# calculate maximum number of "reviews" per venue for the whole dataset
review_business.groupBy(review_business.business_id, review_business.business_review_count).count().orderBy('count').collect()

[Row(business_id=u'qj4KHXGgwyutHpvWRLI87g', business_review_count=5, count=1),
 Row(business_id=u'iK6JSngIDcToad8SBl04Fg', business_review_count=35, count=1),
 Row(business_id=u'ZS55tkaRnPR-QSi2i_871w', business_review_count=3, count=2),
 Row(business_id=u'HHgHEtXyIO228Sybr6vdIg', business_review_count=3, count=2),
 Row(business_id=u'akGAR7Kasfr0AuBBOWYV3g', business_review_count=3, count=2),
 Row(business_id=u'roxnZ0kodKkj5iR72UlQSw', business_review_count=3, count=2),
 Row(business_id=u'-RYgx5kQ91vsQQ7RNJ1DQQ', business_review_count=3, count=2),
 Row(business_id=u'h5csTpDRI7_oo1RUOeHOtQ', business_review_count=3, count=2),
 Row(business_id=u'Fs8u8swdxFOmZAZN2Djmag', business_review_count=3, count=2),
 Row(business_id=u'POcKs2JOkWdq0mm_vtIehw', business_review_count=3, count=2),
 Row(business_id=u'FRqzMYKxASVDhn0W2limWg', business_review_count=3, count=2),
 Row(business_id=u'JoZjCxjbS-01rjBz3KR8Nw', business_review_count=3, count=2),
 Row(business_id=u'5a3ZWk3F9JgSPp97McF76Q', busines

In [14]:
%%time
import time
start_time = time.time()
_review_business = review_business.toPandas()
print("--- %s seconds ---" % (time.time() - start_time))

Py4JJavaError: An error occurred while calling o75.collectToPython.
: java.lang.OutOfMemoryError: GC overhead limit exceeded
	at org.apache.spark.sql.execution.SparkPlan$$anon$1.next(SparkPlan.scala:261)
	at org.apache.spark.sql.execution.SparkPlan$$anon$1.next(SparkPlan.scala:257)
	at scala.collection.Iterator$class.foreach(Iterator.scala:893)
	at org.apache.spark.sql.execution.SparkPlan$$anon$1.foreach(SparkPlan.scala:257)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeCollect$1.apply(SparkPlan.scala:279)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeCollect$1.apply(SparkPlan.scala:278)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:186)
	at org.apache.spark.sql.execution.SparkPlan.executeCollect(SparkPlan.scala:278)
	at org.apache.spark.sql.Dataset$$anonfun$collectToPython$1.apply$mcI$sp(Dataset.scala:2803)
	at org.apache.spark.sql.Dataset$$anonfun$co

## Error msg from Terminal
#### Exception in thread "refresh progress" Exception in thread "dispatcher-event-loop-3" java.lang.OutOfMemoryError: GC overhead limit exceeded
	at org.apache.spark.ui.ConsoleProgressBar.org$apache$spark$ui$ConsoleProgressBar$$refresh(ConsoleProgressBar.scala:69)
	at org.apache.spark.ui.ConsoleProgressBar$$anon$1.run(ConsoleProgressBar.scala:55)
	at java.util.TimerThread.mainLoop(Timer.java:555)
	at java.util.TimerThread.run(Timer.java:505)
java.lang.OutOfMemoryError: GC overhead limit exceeded
	at java.util.concurrent.Executors$DefaultThreadFactory.newThread(Executors.java:612)
	at org.spark_project.guava.util.concurrent.ThreadFactoryBuilder$1.newThread(ThreadFactoryBuilder.java:162)
	at java.util.concurrent.ThreadPoolExecutor$Worker.<init>(ThreadPoolExecutor.java:619)
	at java.util.concurrent.ThreadPoolExecutor.addWorker(ThreadPoolExecutor.java:932)
	at java.util.concurrent.ThreadPoolExecutor.processWorkerExit(ThreadPoolExecutor.java:1025)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1167)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)





# Retrieving big data from RDD to local machine

## abuse RDD.Collect() 
Collect (Action) - Return all the elements of the dataset as an array at the driver program. This is usually useful after a filter or other operation that returns a sufficiently small subset of the data.

In [16]:
%%time
review_business.collect()    

Py4JJavaError: An error occurred while calling o75.collectToPython.
: java.lang.OutOfMemoryError: GC overhead limit exceeded


### Error before imcreasing driver maxResultSize 
####  size serialized results (1035.2 MB) is bigger than spark.driver.maxResultSize (1024.0 MB)
Problem stems from RDD sends/collects (big chunk of) all the data (from different nodes) to the driver node.

### Easy temp fix is changing driver memory in spark instance. 
change spark.driver.maxResultSize by 
1. start pyspark w/ --conf spark.driver.maxResultSize=2g or
2. see next cell

In [17]:
# from pyspark import SparkConf, SparkContext

## In Jupyter you have to stop the current context first
# spark.stop()

In [18]:
# del spark

In [19]:
## restart w/ driver.maxResultSize
# spark = SparkSession \
#         .builder \
#         .master("yarn") \
#         .appName("testing") \
#         .config("spark.executor.instances", "4") \
#         .config("spark.executor.memory","2g") \
#         .config("spark.driver.memory","4g") \
#         .config("spark.executor.cores",'1') \
#         .config("spark.scheduler.mode","FIFO") \
#         .getOrCreate()
##         .config("spark.driver.maxResultSize", "2g") \

# ok well.. encountered some error while re-importing SparkSeesion even tho already "del spark" ... 
### will restart pyspark and specific that in terminal

pyspark --master yarn --deploy-mode client --num-executors 4 --executor-memory 2g --driver-memory 4g --conf spark.driver.maxResultSize=3g

In [20]:
%%time
# review_business.collect()     <<already executed above after we restart

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 14.1 µs


### Error after increasing driver maxResultSize:
Py4JJavaError: An error occurred while calling o75.collectToPython.
: java.lang.OutOfMemoryError: GC overhead limit exceeded

### alternatives?
1. review_business.rdd      # and do all the magic stuff on the cluster, not in the driver
2. Use spark.rdd.compress to compress RDDs when you collect them        << configuration
3. Try to collect it using pagination. 

see https://stackoverflow.com/questions/31058504/spark-1-4-increase-maxresultsize-memory

## recall RDD

Collection of data partitions called RDD. These RDD must follow few properties:

- Immutable,
- Fault Tolerant,
- Distributed,
- +...

Here RDD is either structured or unstructured.

In [15]:
yelp_rb = review_business.rdd

In [16]:
type(yelp_rb)

pyspark.rdd.RDD

In [24]:
yelp_rb.collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: java.lang.IllegalStateException: SparkContext has been shutdown
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2014)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2043)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2062)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2087)
	at org.apache.spark.rdd.RDD$$anonfun$collect$1.apply(RDD.scala:936)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:362)
	at org.apache.spark.rdd.RDD.collect(RDD.scala:935)
	at org.apache.spark.api.python.PythonRDD$.collectAndServe(PythonRDD.scala:458)
	at org.apache.spark.api.python.PythonRDD.collectAndServe(PythonRDD.scala)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at su

## Error this time, after running review_business.collect(), which was executed after we have increased the maxResultSize() --> 
"Py4JJavaError: An error occurred while calling o75.collectToPython. : java.lang.OutOfMemoryError: GC overhead limit exceeded"
### Error following last cell: java.lang.IllegalStateException: SparkContext has been shutdown

In [26]:
spark

In [28]:
sc = SparkContext()   

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=PySparkShell, master=yarn) created by <module> at /Users/admin/anaconda/lib/python2.7/site-packages/IPython/utils/py3compat.py:288 

## Great.. can't even start it again

In [35]:
SparkContext.stop(spark)
# spark.stop()

TypeError: unbound method stop() must be called with SparkContext instance as first argument (got SparkSession instance instead)

### also can't stop.... will restart pyspark.... 
----------------------------

### after restart pyspark ..............

In [23]:
%%time
spark.sparkContext.getConf().getAll() 

CPU times: user 20.1 ms, sys: 5.49 ms, total: 25.6 ms
Wall time: 50.2 ms


[(u'spark.executor.memory', u'2g'),
 (u'spark.executor.instances', u'4'),
 (u'spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_URI_BASES',
  u'http://dhcp-ccc-10261.redrover.cornell.edu:8088/proxy/application_1509845828116_0007'),
 (u'spark.driver.memory', u'4g'),
 (u'spark.driver.port', u'58591'),
 (u'spark.driver.host', u'10.144.40.21'),
 (u'spark.executor.id', u'driver'),
 (u'spark.app.name', u'PySparkShell'),
 (u'spark.ui.proxyBase', u'/proxy/application_1509845828116_0007'),
 (u'spark.app.id', u'application_1509845828116_0007'),
 (u'spark.master', u'yarn'),
 (u'spark.driver.appUIAddress', u'http://10.144.40.21:4040'),
 (u'spark.sql.catalogImplementation', u'hive'),
 (u'spark.rdd.compress', u'True'),
 (u'spark.executorEnv.PYTHONPATH',
  u'/Users/admin/anaconda/lib/python2.7/site-packages/pyspark/python/lib/py4j-0.10.4-src.zip:/Users/admin/anaconda/lib/python2.7/site-packages/pyspark/python/:/Users/admin/gildas-exe-sep15a/x86_64-darwin-gfortran/python:/Us

### 3. pagination

In [24]:
%%time
yelp_rb.first()

CPU times: user 18.2 ms, sys: 13.2 ms, total: 31.4 ms
Wall time: 1min 28s


Row(business_id=u'--9e1ONYQuAa-CB_Rrw7Tw', cool=0, date=u'2012-03-29', funny=0, review_id=u'OJTjgyV6HYRPo7ZO1KlRsQ', business_avg_stars=5, text=u'For me the bar has been raised! \n\nI thought the Conch Republic in Key West was my best meal ever. It has been topped. The best Ribeye I have ever eaten. Everything about this place was top shelf. I had the escargot, Ribeye, loaded baked potato, and finished it off with the best pecan pie ever.', useful=0, user_id=u'poUDFITGDHUoaiAvwFzXEw', address=u'3355 Las Vegas Blvd S', business_type=Row(AcceptsInsurance=None, AgesAllowed=None, Alcohol=u'full_bar', Ambience=Row(casual=False, classy=True, divey=False, hipster=False, intimate=False, romantic=False, touristy=False, trendy=False, upscale=True), BYOB=False, BYOBCorkage=u'yes_corkage', BestNights=None, BikeParking=False, BusinessAcceptsBitcoin=None, BusinessAcceptsCreditCards=True, BusinessParking=Row(garage=True, lot=False, street=False, valet=True, validated=False), ByAppointmentOnly=None, C

In [26]:
yelp_rb.take(2)

[Row(business_id=u'--9e1ONYQuAa-CB_Rrw7Tw', cool=0, date=u'2012-03-29', funny=0, review_id=u'OJTjgyV6HYRPo7ZO1KlRsQ', business_avg_stars=5, text=u'For me the bar has been raised! \n\nI thought the Conch Republic in Key West was my best meal ever. It has been topped. The best Ribeye I have ever eaten. Everything about this place was top shelf. I had the escargot, Ribeye, loaded baked potato, and finished it off with the best pecan pie ever.', useful=0, user_id=u'poUDFITGDHUoaiAvwFzXEw', address=u'3355 Las Vegas Blvd S', business_type=Row(AcceptsInsurance=None, AgesAllowed=None, Alcohol=u'full_bar', Ambience=Row(casual=False, classy=True, divey=False, hipster=False, intimate=False, romantic=False, touristy=False, trendy=False, upscale=True), BYOB=False, BYOBCorkage=u'yes_corkage', BestNights=None, BikeParking=False, BusinessAcceptsBitcoin=None, BusinessAcceptsCreditCards=True, BusinessParking=Row(garage=True, lot=False, street=False, valet=True, validated=False), ByAppointmentOnly=None, 

In [29]:
%%time
yelp_rb.toDF().show()

ValueError: Some of types cannot be determined by the first 100 rows, please try again with sampling

## don't seem to have a straight way out, can only do:
- buy more computing power
- filter the data to smaller size before doing actions 

# merge, join, union transformation

e.g., Union()
- Union is basically used to merge two RDDs together if they have the same structure.

In [41]:
%%time
review.union(bus).collect()

AnalysisException: u"Union can only be performed on tables with the same number of columns, but the first table has 9 columns and the second table has 15 columns;;\n'Union\n:- Repartition 150, true\n:  +- Relation[business_id#86,cool#87L,date#88,funny#89L,review_id#90,stars#91L,text#92,useful#93L,user_id#94] json\n+- Repartition 150, true\n   +- Relation[address#46,attributes#47,business_id#48,categories#49,city#50,hours#51,is_open#52L,latitude#53,longitude#54,name#55,neighborhood#56,postal_code#57,review_count#58L,stars#59,state#60] json\n"

---

### glom - return an RDD created by coalescing all elements within each partition into a list.

http://parrotprediction.com/partitioning-in-apache-spark/

Spark uses different partitioning schemes for various types of RDDs and operations. In a case of using parallelize(), data is evenly distributed between partitions using their indices (no partitioning scheme is used). << how could I get sc.parallelize() with my current setup tho....

In [42]:
print("Number of partitions: {}".format(yelp_rb.getNumPartitions()))

Number of partitions: 200


In [43]:
print("Partitioner: {}".format(yelp_rb.partitioner))
print("Partitions structure: {}".format(yelp_rb.glom().collect()))

Partitioner: None


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Total size of serialized results of 106 tasks (3.0 GB) is bigger than spark.driver.maxResultSize (3.0 GB)
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1499)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1487)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1486)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1486)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:814)
	at org.apache.spark.scheduler.DAGScheduler$$anonf

### again .... error 
Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Total size of serialized results of 106 tasks (3.0 GB) is bigger than spark.driver.maxResultSize (3.0 GB)

In [18]:
sc.stop()


In [19]:
%%time
# in the following, let's do simple data instead of hauling the big yelp data
nums = range(0, 10)

with SparkContext("local") as sc:
    rdd = sc.parallelize(nums)
    
    print("Number of partitions: {}".format(rdd.getNumPartitions()))
    print("Partitioner: {}".format(rdd.partitioner))
    print("Partitions structure: {}".format(rdd.glom().collect()))


Number of partitions: 1
Partitioner: None
Partitions structure: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]
CPU times: user 20 ms, sys: 9.7 ms, total: 29.7 ms
Wall time: 2.01 s


In [21]:
%%time
# allow driver to use two local cores.
with SparkContext("local[2]") as sc:
    rdd = sc.parallelize(nums)
    
    print("Default parallelism: {}".format(sc.defaultParallelism))
    print("Number of partitions: {}".format(rdd.getNumPartitions()))
    print("Partitioner: {}".format(rdd.partitioner))
    print("Partitions structure: {}".format(rdd.glom().collect()))


Default parallelism: 2
Number of partitions: 2
Partitioner: None
Partitions structure: [[0, 1, 2, 3, 4], [5, 6, 7, 8, 9]]
CPU times: user 16.3 ms, sys: 6.02 ms, total: 22.3 ms
Wall time: 1.43 s


In [22]:
%%time
# what happens if number of partitions exceeds number of data records?
with SparkContext("local") as sc:
    rdd = sc.parallelize(nums, 15)
    
    print("Number of partitions: {}".format(rdd.getNumPartitions()))
    print("Partitioner: {}".format(rdd.partitioner))
    print("Partitions structure: {}".format(rdd.glom().collect()))

Number of partitions: 15
Partitioner: None
Partitions structure: [[], [0], [1], [], [2], [3], [], [4], [5], [], [6], [7], [], [8], [9]]
CPU times: user 20.9 ms, sys: 7.46 ms, total: 28.3 ms
Wall time: 1.59 s


--> Spark created requested a number of partitions but most of them are empty. This is bad because the time needed to prepare a new thread for processing data (one element) is significantly greater than processing time itself (you can analyze it in Spark UI).

## Custom partitions with partitionBy()

partitionBy() *transformation* allows applying custom partitioning logic over the RDD, requires data to be in key/value format we will need to transform the data.

In [23]:
with SparkContext("local[2]") as sc:
    rdd = sc.parallelize(nums) \     # put into RDD w/o partitioning schmee
        .map(lambda el: (el, el)) \ 
        .partitionBy(2) \           # split into 2 chunks using the default hash partitioner
        .persist()                  # persist storage level, default = memory
    
    print("Number of partitions: {}".format(rdd.getNumPartitions()))
    print("Partitioner: {}".format(rdd.partitioner))
    print("Partitions structure: {}".format(rdd.glom().collect()))

Number of partitions: 2
Partitioner: <pyspark.rdd.Partitioner object at 0x116ac4590>
Partitions structure: [[(0, 0), (2, 2), (4, 4), (6, 6), (8, 8)], [(1, 1), (3, 3), (5, 5), (7, 7), (9, 9)]]


In [27]:
yelp_rb.persist?

In [30]:
%%time
# santity
from pyspark.rdd import portable_hash
 
num_partitions = 2
 
for el in nums:
    # partition number is evaluated using: partition = partitionFunc(key) % num_partitions.
    # default partition func in pyspark = hash
    print("Element: [{}]: {} % {} = partition {}".format(
        el, portable_hash(el), num_partitions, portable_hash(el) % num_partitions))

Element: [0]: 0 % 2 = partition 0
Element: [1]: 1 % 2 = partition 1
Element: [2]: 2 % 2 = partition 0
Element: [3]: 3 % 2 = partition 1
Element: [4]: 4 % 2 = partition 0
Element: [5]: 5 % 2 = partition 1
Element: [6]: 6 % 2 = partition 0
Element: [7]: 7 % 2 = partition 1
Element: [8]: 8 % 2 = partition 0
Element: [9]: 9 % 2 = partition 1
CPU times: user 236 µs, sys: 53 µs, total: 289 µs
Wall time: 249 µs


#### slightly more realistic example

In [31]:
# data consist of various dummy transactions made across different countries.

transactions = [
    {'name': 'Bob', 'amount': 100, 'country': 'United Kingdom'},
    {'name': 'James', 'amount': 15, 'country': 'United Kingdom'},
    {'name': 'Marek', 'amount': 51, 'country': 'Poland'},
    {'name': 'Johannes', 'amount': 200, 'country': 'Germany'},
    {'name': 'Paul', 'amount': 75, 'country': 'Poland'},
]

#### To optimize network traffic for further analysis, put records from one country in one node.
Will use custom_partitioner(), which returns an integer for given object (tuple key).


In [40]:
hash?

In [38]:
# ensure that data for each country is in one partition
def country_partitioner(country):
    return hash(country)
 
# Validate results to see what partitions are assigned for each country.
num_partitions = 5
 
print(country_partitioner("Poland") % num_partitions)
print(country_partitioner("Germany") % num_partitions)
print(country_partitioner("United Kingdom") % num_partitions)

AttributeError: 'NoneType' object has no attribute '_jvm'

In [42]:

with SparkContext("local[2]") as sc:
    rdd = sc.parallelize(transactions) \
        .map(lambda el: (el['country'], el)) \
        .partitionBy(4, country_partitioner)
    
    print("Number of partitions: {}".format(rdd.getNumPartitions()))
    print("Partitioner: {}".format(rdd.partitioner))
    print("Partitions structure: {}".format(rdd.glom().collect()))

Number of partitions: 4
Partitioner: <pyspark.rdd.Partitioner object at 0x116cbf810>


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 0.0 failed 1 times, most recent failure: Lost task 1.0 in stage 0.0 (TID 1, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/admin/anaconda/lib/python2.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 177, in main
    process()
  File "/Users/admin/anaconda/lib/python2.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 172, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/admin/anaconda/lib/python2.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 138, in dump_stream
    for obj in iterator:
  File "/Users/admin/anaconda/lib/python2.7/site-packages/pyspark/rdd.py", line 1753, in add_shuffle_key
    buckets[partition

### some errors to look into later..

 ------------------------------------------------------
 *** side question *** 
 
While running stuff in notebook, the following appears in the terminal:   
[W 22:25:09.065 NotebookApp] Timeout waiting for kernel_info reply from 5e476d4b-3c33-4eee-839b-e3173fda7970

17/11/04 22:25:39 WARN metastore.ObjectStore: Failed to get database global_temp, returning NoSuchObjectException

[Stage 6:=================>                                      (47 + 2) / 150][I 22:26:45.828 NotebookApp] Saving file at /CostlyFunctions.ipynb

[Stage 9:>                                                          (0 + 2) / 2][I 22:27:00.840 NotebookApp] Saving file at /CostlyFunctions.ipynb

17/11/04 22:27:03 WARN util.Utils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.

**what are the stages???? and what do they mean??**